# Пример использования модуля MultiTankMaterialBalance

<a id='index-0'></a>

## Содержание

- [Пример использования модуля MultiTankMaterialBalance](#Пример-использования-модуля-MultiTankMaterialBalance)
    - [Введение](#Введение)
    - [Основная идея](#Основная-идея)
        - [Связь с CRM-моделью](#Связь-с-CRM-моделью)
    - [Исходные данные для моделирования](#Исходные-данные-для-моделирования)
    - [Решение прямой задачи](#Решение-прямой-задачи)
        - [Решение системы линейных уравнений](#Решение-системы-линейных-уравнений)
    - [Параметры модели и целевая функция](#Параметры-модели-и-целевая-функция)
        - [Способы масштабирования параметров](#Способы-масштабирования-параметров)

## Введение

[Материальный баланс](https://petrowiki.spe.org/Material_balance_in_oil_reservoirs) (МБ) в нефтепромысловом деле - простейшая форма динамической модели нефтяного или газового месторождения. Это простая концепция, подчиняющаяся закону сохранения масс, согласно которому извлечённый объём равен сумме изменения первоначального и привнесённого объёмов в пласте:

<a id='simple-matbal'></a>
$$
V_\mathrm{извлеченный}=\Delta V_\mathrm{первоначальный} + V_\mathrm{привнесённый}
$$

Недостатком классической модели МБ является то, что это по сути нульмерная модель всей залежи, не учитывающая распространение флюидов в пространстве.

Мы представим основное уравнение МБ в немного модифицированном виде, более похожим на те, которые используются при численном моделировании пластовых систем:

<a id='classic-matbal'></a>
$$
\frac{1}{\Delta t^n} \sum_{l = \mathrm w, \, \mathrm o} \left( S_l^{n+1} \frac{V_\mathrm p^{n+1}}{B_l^{n+1}} - S_l^{n} \frac{V_\mathrm p^{n}}{B_l^{n}} \right) + 
\sum_{l = \mathrm w, \, \mathrm o} q_l^{n+1} - q_{\mathrm{inj}}^{n+1} - q_{\mathrm{aq}}^{n+1} = 0, \tag{1}
$$

где $ \Delta t $ - размер шага по времени (обычно месяц); $ S_l, B_l, q_l $ - насыщенность, объемный коэффицент, отбор $ l $-го флюида соответственно ($ l=\mathrm w$ - вода, $ l=\mathrm o$ - нефть); $ V_\mathrm p $ - поровый объем; $ q_{\mathrm{inj}} $ - закачка флюида в пласт; $ q_{\mathrm{aq}} $ - приток из аквифера; $ n $ - номер шага.

Причем $ S_\mathrm w + S_\mathrm o = 1 $.

Для расчета $ q_{\mathrm{aq}} $ используются различные аналитические [модели аквиферов](https://petrowiki.spe.org/Water_influx_models).

Подробное описание классической модели МБ и моделей аквиферов можно найти в <a href=#Ahm10 id=Ahm10-link>[Ahm10]</a>, <a href=#Dak01 id=Dak01-link>[Dak01]</a>, <a href=#Gol82 id=Gol82-link>[Gol82]</a>. Связь между классическим МБ и численным моделированием пластовых систем можно найти в <a href=#EAKK01 id=EAKK01-link>[EAKK01]</a>.

In [ ]:
using MultiTankMaterialBalance
using CairoMakie
using Makie.MakieLayout: get_labeled_plots
using PrettyTables
using Statistics
using FiniteDiff
using Test
using NLopt
import UnicodePlots

# Максимальный кол-во выводимых столбцов DataFrame
ENV["COLUMNS"] = 220

# Используемая разрядность
Float = Float64

## Основная идея

В разработанном модуле исходный пласт представляется как набор блоков, для каждого из которых справедливы те же уравнения, что и в классической модели МБ. Единственное отличие от классической модели МБ в том, что здесь дополнительно учитываются перетоки между блоками. Более того, сам аквифер рассматривается как обычный блок без истории отборов (скважин).

Ниже показан пример такой модели. Каждый блок содержит по одной скважине (`W1` ... `W9`). Причем скважины `W3` и `W5` являются нагнетательными. Остальные скважины - добывающие. Блоки, содержащие скважины `W1`, `W2`, `W3`, `W5`, `W7`, `W8` и `W9` граничат с блоком для аквифера `AQ`, который тут не показан.

![Example of multiple tanks containing single wells](welltanks.png "Example of multiple tanks containing single wells")

Таким образом, уравнение [(1)](#classic-matbal) принимает векторный вид

<a id='multi-tank-matbal'></a>
$$
\frac{1}{\Delta t^n} \sum_{l = \mathrm w, \, \mathrm o} 
\left( \mathbf S_l^{n+1} \circ \frac{\mathbf V_\mathrm p^{n+1}}{\mathbf B_l^{n+1}} - 
\mathbf S_l^{n} \circ \frac{\mathbf V_\mathrm p^{n}}{\mathbf B_l^{n}} \right) + 
\mathbf C^\mathrm T \left( \mathbf T \circ \mathbf C \mathbf P^{n+1} \right) + 
\mathbf T_{\mathrm{bnd}} \circ \left( \mathbf P^{n+1} - \mathbf P_\mathrm i \right) +
\sum_{l = \mathrm w, \, \mathrm o} \mathbf q_l^{n+1} - 
\lambda \circ \mathbf q_{\mathrm{inj}}^{n+1} = 0, \tag{2}
$$

где $ \mathbf P $ - вектор текущего пластового давления в блоках; $ \mathbf P_\mathrm i $ - вектор начального пластового давления в блоках; $ \mathbf C $ - матрица смежности блоков; $ \mathbf T $ - вектор межблочных проводимостей; $ \mathbf T_{\mathrm{bnd}} $ - вектор проводимости для блоков, связанных с границей постоянного давления; $ \lambda $ - вектор коэффициентов эффективности закачки; символ `∘` обозначает поэлементное умножение векторов ([произведение Адамара](https://en.wikipedia.org/wiki/Hadamard_product_(matrices))).

Начальным условием является $ \mathbf P^{0} = \mathbf P_\mathrm i $. Граничные условия заданы через члены источников/стоков (упрощенные модели скважин).

Матрица $ \mathbf C $ имеет специальную структуру, которая позволяет легко реализовать дискретные операторы градиента и дивергенции (см. <a href=#Lie19 id=Lie19-link>[Lie19]</a>):

- градиент: $ \operatorname{grad}(\mathbf x) = \mathbf C \mathbf x $
- дивергенция: $ \operatorname{div}(\mathbf x) = - \mathbf C^\mathrm T \mathbf x $

Члены, зависящие от давления:

<a id='porous-volume'></a>
$$ 
\mathbf V_\mathrm p^{n+1} = \mathbf V_\mathrm{pi} \circ e^{\mathbf c_\mathrm f \circ \left( \mathbf P^{n+1} - \mathbf P_\mathrm i \right)} \tag{3}
$$
<a id='volume-factor'></a>
$$ 
\mathbf B_l^{n+1} = \mathbf B_{l\mathrm{i}} \circ e^{- \mathbf c_l \circ \left( \mathbf P^{n+1} - \mathbf P_\mathrm i \right)} \tag{4}
$$

где $ \mathbf V_\mathrm{pi} $, $ \mathbf B_{l\mathrm{i}} $ и $ \mathbf P_\mathrm i $,  - начальные поровый объем, объемный коэффициет и пластовое давление соответственно; $ \mathbf c_\mathrm f $ и $ \mathbf c_l $ - сжимаемости порового пространства и флюида соответственно; $ l = \mathrm w,\, \mathrm o $.

### Связь с CRM-моделью

Модель многоблочного МБ тесно связана с так называемой [CRM-моделью](https://www.sciencedirect.com/science/article/abs/pii/S0920410509002046).

Аккумулятивный член приближенно можно представить как

$$
\sum_{l = \mathrm w, \, \mathrm o} 
\left( \left( S_l \frac{V_\mathrm p}{B_l} \right)_i^{n+1} - 
\left( S_l \frac{V_\mathrm p}{B_l} \right)_i^{n} \right) \approx
c_{\mathrm t,\, i} V_{\mathrm{pi},\, i} (P^{n+1}_i - P^n_i)
$$

где $ c_{\mathrm t} = c_{\mathrm f} + S_{\mathrm{wi}} c_{\mathrm w} + \left( 1 - S_{\mathrm{wi}} \right) c_{\mathrm o} $ - общая сжимаемость системы; $ c_{\mathrm f} $, $ c_{\mathrm w} $ и $ c_{\mathrm o} $ - сжимаемости пор, воды и нефти соответственно; $ V_{\mathrm{pi}} $ - начальный поровый объем; $ S_{\mathrm{wi}} $ - начальная водонасыщенность; $ i $ - номер блока.

При известной продуктивности $ J_{\mathrm p} $ , забойном давлении $ P_{\mathrm{bhp}} $ и добыче жидкости $ q_{\mathrm{liq}} $ добывающей скважины

$$
c_{\mathrm t,\, i} V_{\mathrm{pi},\, i} (P^{n+1}_i - P^n_i) =
\underbrace{\frac{c_{\mathrm t,\, i} V_{\mathrm{pi},\, i}}{J_{\mathrm p,\, i}}}_{\tau_i}
\left( q^{n+1}_{\mathrm{liq},\, i} - q^n_{\mathrm{liq},\, i} \right) +
\underbrace{c_{\mathrm t,\, i} V_{\mathrm{pi},\, i}}_{\tau_i J_{\mathrm p,\, i}}
\left( P^{n+1}_{\mathrm{bhp},\, i} - P^n_{\mathrm{bhp},\, i} \right)
$$

Таким образом, модель многоблочного МБ можно представить как CRM-модель, где

- постоянная времени $ \tau = \frac{c_{\mathrm t} V_{\mathrm{pi}}}{J_{\mathrm p}} $;
- коэффициент связности $ f_{ij} = \frac{T_{ij}}{\sum_{j \in \omega_i} T_{ij}} $.

Однако фундаментальные отличия многоблочного МБ от CRM-модели


| Модель          | Известно             | Надо найти           |
| :-------------: | :------------------: | :------------------: |
| Многоблочный МБ | $ q_{\mathrm{liq}} $ | $ P $                |
| CRM-модель      | $ P_{\mathrm{bhp}} $ | $ q_{\mathrm{liq}} $ |

## Исходные данные для моделирования

Для работы модуля, реализующего многоблочный МБ, требуются исходные данные:

- показатели работы скважин:
    - история добычи воды (`Qwat`) и нефти (`Qoil`), а также закачки воды (`Qinj`);
    - динамика забойных давлений (`Pbhp_prod`, `Pbhp_inj`);
    - замеры пластового давления (`Pres`);
- параметры блоков:
    - начальное пластовое давление (`Pi`);
    - начальный поровый объем (`Vpi`);
    - начальная насыщенность водой (`Swi`);
    - сжимаемость порового пространства (`cf`):
    - PVT-свойства флюидов (`cw`, `Bw`, `co`, `Bo`);
    - межблочные проводимости (`Tconn`);
    - проводимость на границе постоянного давления (`Tconst`);
- параметры скважин:
    - коэффициент продуктивности для добывающих скважин (`Prod_index`);
    - коэффициент приемистости для нагнетательных скважин (`Inj_index`);
    - коэффициент эффективности закачки для нагнетательных скважин (`Frac_inj`);
- ограничения на возможные значения пластового давления в блоках:
    - минимально возможное пластовое давление (`Pres_min`);
    - максимально возможное пластовое давление (`Pres_max`).

In [ ]:
# Путь к csv-файлам с исходными данными
workdir = "../data";
opts_csv = Dict("dateformat" => "dd.mm.yyyy", "delim" => ";")

# Исходные данные для расчета
df_rates = read_rates(joinpath(workdir, "tank_prod.csv"), opts_csv)
df_params = read_params(joinpath(workdir, "tank_params.csv"), opts_csv)

# Дополнительная предобработка параметров
process_params!(df_params, df_rates);

In [ ]:
# Структура таблицы с параметрами
df_params

In [ ]:
# Список всех параметров блоков
println(unique(df_params.Parameter))

In [ ]:
# Пример истории работы скважины W7
df_rates[df_rates.Tank .== "W7", :]

In [ ]:
# Проведенные ГТМ на скважинах
df_rates[.!ismissing.(df_rates.Wellwork), :]

In [ ]:
# Проведенные ГДИС на скважинах
df_rates[.!ismissing.(df_rates.Pres), :]

In [ ]:
function plot_tanks(tanks, plots, dates; ncol=3, resolution=(600, 400), date_div=6)
    
    # Обозначение меток временной шкалы
    tempo = string.(unique(dates))
    lentime = length(tempo)
    slice_dates = range(1, lentime, step=lentime ÷ date_div)
    
    f = Figure(; resolution)    
    
    for (i, tank) ∈ enumerate(tanks)
        row, col = (i - 1) ÷ ncol + 1, (i - 1) % ncol + 1
        
        # Левая ось для дебитов
        ax1 = Axis(f[row, col], ylabel="Rate, m3/day", title=tank)
        xlims!(ax1, 1, lentime)
        ylims!(ax1, low=0)

        # Правая ось для давлений
        ax2 = Axis(f[row, col], ylabel="Pressure, bar", yaxisposition = :right, ygridvisible=false)
        xlims!(ax2, 1, lentime)
        ylims!(ax2, low=0)
        hidespines!(ax2)
        hidexdecorations!(ax2)
        
        # Размещение меток для временной шкалы
        ax1.xticks = (slice_dates, tempo[slice_dates])
        ax1.xticklabelrotation = π/4
        ax1.xticklabelalign = (:right, :center)
        ax1.xticklabelsize = 14
        
        for (lbl, plt) ∈ zip(keys(plots), values(plots))
            ax = plt.axis == 1 ? ax1 : ax2
            data = plt.getter(tank)
            plt.plot(ax, data; label=string(lbl), plt.attrs...)
        end
    end
    
    # Функция 'contents' вытаскивает объекты 'Axis' для левой и правой осей 
    # Функция 'get_labeled_plots' вытаскивает из 'Axis' промаркированные объекты графиков 'plots'
    plots1, labels1 = get_labeled_plots(contents(f[1, 1])[1]; merge=false, unique=false)
    plots2, labels2 = get_labeled_plots(contents(f[1, 1])[2]; merge=false, unique=false)

    # Общая легенда для всех графиков
    maxrow = (length(tanks) - 1) ÷ ncol + 1
    Legend(f[maxrow+1, 1:ncol], 
        [plots1..., plots2...],
        [labels1..., labels2...],
        orientation=:horizontal, 
        tellwidth=false, 
        tellheight=true
    )
    
    return f
end

# Набор уникальных дат и блоков
dates = unique(df_rates.Date)
tanks = unique(df_rates.Tank)

# Графики по блокам
data_for_plots = (
    Qliq = (getter = tank -> df_rates[df_rates.Tank .== tank, :Qliq], 
            axis = 1, plot = lines!, 
            attrs = (color=:green, linewidth=3)),
    Qinj = (getter = tank -> df_rates[df_rates.Tank .== tank, :Qinj], 
            axis = 1, plot = lines!, 
            attrs = (color=:blue, linewidth=3)),
    Pbhp = (getter = tank -> df_rates[df_rates.Tank .== tank, :Pbhp_prod], 
            axis = 2, plot = scatter!, 
            attrs = (marker=:diamond, markersize=12, color=:violet)),
    Pinj = (getter = tank -> df_rates[df_rates.Tank .== tank, :Pbhp_inj], 
            axis = 2, plot = scatter!, 
            attrs = (marker=:diamond, markersize=12, color=:darkviolet)),
    Pres = (getter = tank -> df_rates[df_rates.Tank .== tank, :Pres], 
            axis = 2, plot = scatter!,
            attrs = (marker=:circle, markersize=15, color=:red)),
)

f = plot_tanks(tanks[2:end], data_for_plots, dates; resolution=(2000, 1000), ncol=3)
f

## Решение прямой задачи

Представим уравнение [(2)](#multi-tank-matbal) в виде следующей системы уравнений

<a id='nonlinear-system'></a>
$$
\mathbf f^{n+1} \equiv \mathbf f^{n+1} \left( \mathbf P^{n+1},\, \mathbf P^n,\, \mathbf m \right) = \mathbf 0 \tag{3}
$$

где $\mathbf m$ - известные параметры модели.

Уравнения [(3)](#nonlinear-system) являтся нелинейными, поэтому для их решения в модуле реализован [метод Ньютона-Рафсона](https://en.wikipedia.org/wiki/Newton%27s_method). В соответствии с данным методом для каждого временного шага $ \Delta t^n $ итерационно решается система линейных уравнений и обновляется вектор давления

<a id='newton-method'></a>
$$
\left[ \nabla_{\mathbf P^{n+1}} \mathbf f^{n+1} \right]^{\nu} \delta \mathbf P^{n+1,\, \nu+1} = -\mathbf f^{n+1,\, \nu} \tag{4}
$$

<a id='newton-update'></a>
$$
\mathbf P^{n+1,\, \nu+1} = \mathbf P^{n+1,\, \nu} + \delta \mathbf P^{n+1,\, \nu+1} \tag{5}
$$

где $ \nu $ - номер итерации; $ \mathbf P^{n+1,\, 0} = \mathbf P^{n}  $.

Данный метод требует вычисления якобиана системы, который в модуле рассчитывается аналитически

<a id='system-jacobian'></a>
$$
\left[ \nabla_{\mathbf P^{n+1}} \mathbf f^{n+1} \right] = \frac{1}{\Delta t^{n}} \sum_{l = \mathrm w, \, \mathrm o} 
\operatorname{\mathbf D} \left[ \left( \mathbf c_l + \mathbf c_{\mathrm f} \right) \circ 
\mathbf S_l^{n+1} \circ \frac{\mathbf V_{\mathrm p}^{n+1}}{\mathbf B_l^{n+1}} \right] +
\operatorname{\mathbf D} \left[ \mathbf T_{\mathrm{bnd}} \right] + 
\mathbf C^\mathrm T \operatorname{\mathbf D} \left[ \mathbf T \right] \mathbf C \tag{6}
$$

где $ \operatorname{\mathbf D} \left[ \mathbf x \right] $ - диагональная матрица с элементами вектора $ \mathbf x $ на главной диагонали.

Можно заметить, что только элементы на главной диагонали якобиана [(6)](#system-jacobian) зависят от давления $\mathbf P$. Остальные ненулевые элементы вне главной диагонали якобиана размещаются в соответствии с произведением $\mathbf C^{\mathrm T} \mathbf C$, поэтому якобиан является симметричным.

In [ ]:
# Описание прямой задачи
prob = NonlinearProblem{Float}(df_rates, df_params)

# Структура матрицы смежности блоков
data = hcat(1:size(prob.C, 1), prob.C)
header = vcat("No", "AQ", ["W$n" for n in 1:9])
header_crayon = vcat(Crayon(foreground=:black, bold=true),
                     Crayon(foreground=:blue, bold=true),
                     fill(Crayon(foreground=:red, bold=true), 9)
)
hl_zero = Highlighter((data, i, j) -> (data[i, j] == 0) && (j > 1), 
                      Crayon(foreground=(242, 242, 242)))
hl_one = Highlighter((data, i, j) -> (data[i, j] != 0) && (j > 1), 
                     Crayon(foreground=:black, bold=true, background=(255, 204, 0)))
hl_num = Highlighter((data, i, j) -> j == 1, 
                     Crayon(foreground=:black, bold=true))

pretty_table(data;
    title = "Структура матрицы C",
    title_alignment = :C,
    title_crayon = Crayon(foreground=:cyan, bold=true),
    title_same_width_as_table = true,
    header = header,
    header_crayon = header_crayon,
    formatters = ft_printf("%.0f"),
    highlighters = (hl_zero, hl_one, hl_num,)
)

In [ ]:
# Структура якобиана
J = prob.C' * prob.C
UnicodePlots.spy(J; height=15, width=30)

### Решение системы линейных уравнений

Для решения системы линейных уравнений [(4)](#newton-method) в модуле реализованы 3 способа:

- `DenseLinearSolver`: представление якобиана в виде обычной матрицы и решение с помощью [стандартного разложения Холецкого](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.cholesky)
- `RecursiveLinearSolver`: представление якобиана в виде обычной матрицы и решение с помощью [рекурсивной модификации LU-разложения](https://github.com/YingboMa/RecursiveFactorization.jl)
- `SparseLinearSolver`: представление якобиана в виде разреженой матрицы и решение с помощью [специальной версии разложения Холецкого](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.cholesky)

In [ ]:
# Способ решения СЛАУ
linalg = DenseLinearSolver{Float}(prob)

# Алгоритм решения прямой задачи
opts_nsol = Dict("maxiters" => 10, "P_tol" => 1e-7, "r_tol" => 1e-5)
solver = NewtonSolver{Float}(prob, linalg, opts_nsol)

# Решение прямой задачи
solve!(solver; verbose=true);

In [ ]:
# Графики по блокам
data_for_plots = (    
    Pcalc = (getter = tank -> prob.params.Pcalc[parse(Int, tank[2]) + 1, :], 
             axis = 2, plot = lines!, 
             attrs = (color=:black, linewidth=3)),
    data_for_plots...
)

f = plot_tanks(tanks[2:end], data_for_plots, dates; resolution=(2000, 1000), ncol=3)
f

## Параметры модели и целевая функция

Настройка параметров модели осуществляется путем минимизации следующей целевой функции

<a id='target-function'></a>
$$
\beta = \beta_\mathrm{res} + \beta_\mathrm{bhp} + \beta_\mathrm{inj} + \beta_\mathrm{min} + \beta_\mathrm{max} + \beta_\mathrm{L2} \rightarrow \mathrm{min} \tag{7}
$$

где

- член для расчета взвешенного отклонения между расчетными и фактическими значениями пластового давления

<a id='res-term'></a>
$$
\beta_\mathrm{res} = \alpha_\mathrm{res} \sum_{n=1}^L 
\left( \mathbf P^n - \mathbf P_\mathrm{obs}^n \right)^\mathrm T 
\operatorname{\mathbf D} \left[ \mathbf w_\mathrm{res}^n \right] 
\left( \mathbf P^n - \mathbf P_\mathrm{obs}^n \right) \tag{8}
$$

- член для расчета взвешенного отклонения между расчетными и фактическими значениями забойного давления в добывающих скважинах

<a id='bhp-term'></a>
$$
\beta_\mathrm{bhp} = \alpha_\mathrm{bhp} \sum_{n=1}^L 
\left( \mathbf P_\mathrm{bhp}^n - \mathbf P_\mathrm{bhp,\, obs}^n \right)^\mathrm T 
\operatorname{\mathbf D} \left[ \mathbf w_\mathrm{bhp}^n \right] 
\left( \mathbf P_\mathrm{bhp}^n - \mathbf P_\mathrm{bhp,\, obs}^n \right) \tag{9}
$$

- член для расчета взвешенного отклонения между расчетными и фактическими значениями забойного давления в нагнетательных скважинах

<a id='inj-term'></a>
$$
\beta_\mathrm{inj} = \alpha_\mathrm{inj} \sum_{n=1}^L 
\left( \mathbf P_\mathrm{inj}^n - \mathbf P_\mathrm{inj,\, obs}^n \right)^\mathrm T 
\operatorname{\mathbf D} \left[ \mathbf w_\mathrm{inj}^n \right] 
\left( \mathbf P_\mathrm{inj}^n - \mathbf P_\mathrm{inj,\, obs}^n \right) \tag{10}
$$

- член для расчета взвешенного отклонения между расчетными и минимальными значениями пластового давления

<a id='min-term'></a>
$$
\beta_\mathrm{min} = \alpha_\mathrm{min} \sum_{n=1}^L 
\left( \mathbf P^n - \mathbf P_\mathrm{min}^n \right)^\mathrm T 
\operatorname{\mathbf D} \left[ \mathbf w_\mathrm{min}^n \right] 
\left( \mathbf P^n - \mathbf P_\mathrm{min}^n \right) \tag{11}
$$

- член для расчета взвешенного отклонения между расчетными и максимальными значениями пластового давления

<a id='max-term'></a>
$$
\beta_\mathrm{max} = \alpha_\mathrm{max} \sum_{n=1}^L 
\left( \mathbf P^n - \mathbf P_\mathrm{max}^n \right)^\mathrm T 
\operatorname{\mathbf D} \left[ \mathbf w_\mathrm{max}^n \right] 
\left( \mathbf P^n - \mathbf P_\mathrm{max}^n \right) \tag{12}
$$

- член L2-регуляризации

<a id='L2-term'></a>
$$
\beta_\mathrm{L2} = \alpha_\mathrm{L2} \mathbf m^\mathrm T \mathbf m \tag{13}
$$

Параметры $\mathbf m$, которые можно изменять в процессе минимизации целевой функции [(7)](#target-function)

- межблочные проводимости `Tconn`
- проводимость с границей постоянного давления `Tconst`
- начальный поровый объем блока `Vpi`
- сжимаемость порового пространства `cf`
- коэффициент эффективности закачки `λ`

Часть параметров настраиваются автоматически (по результатам расчета пластового давления)

- коэффициент продуктивности добывающих скважин `Jp`
- коэффициент приемистости нагнетательных скважин `Jinj`

### Способы масштабирования параметров

В модуле реализованы два способа масштабирования параметров (для улучшения сходимости оптимизации)

- линейное масштабирование (приводит к оптимизации с простыми ограничениями)

<a id='linear-scale'></a>
$$
\tilde{\mathbf m} = \frac{\mathbf m - \mathbf m_\mathrm{min}}{\mathbf m_\mathrm{max} - \mathbf m_\mathrm{min}}
$$

где $ 0 \le \tilde{\mathbf m} \le 1 $

- сигмоидальное масштабирование (приводит к оптимизации без ограничений)

<a id='sigmoid-scale'></a>
$$
\mathbf z = \frac{\mathbf m - \mathbf m_\mathrm{min}}{\mathbf m_\mathrm{max} - \mathbf m_\mathrm{min}} \\
\tilde{\mathbf m} = \ln \frac{\mathbf z}{\mathbf 1 - \mathbf z}
$$

где $ -\infty \lt \tilde{\mathbf m} \lt +\infty $

In [ ]:
opts_targ = Dict("alpha_resp" => 1, "alpha_bhp" => 0.01, "alpha_inj" => 0.01, "alpha_lb" => 10, "alpha_ub" => 10, "alpha_l2" => 1)

# Способ масштабирования параметров
scale = SigmoidScaling{Float}(df_params)

# Список оптимизируемых параметров
fset = FittingSet{Float}(df_params, prob, scale)

# Целевая функция
targ = TargetFunction{Float}(df_rates, df_params, prob, fset, opts_targ)

# Обновить внутреннее состояние целевой функции
update_targ!(targ)

# Рассчет Кпрод/Кприем
calc_well_index!(targ)

getvalues(targ)

In [ ]:
# Сопоставление измеренных и рассчитанных давлений
function observed_pressure(df, name, tanks, dates, Pcalc; digits=2)
    df_obs = df[.!ismissing.(df[!, name]), :]
    Pobs = df_obs[!, name]
    observed_dates = indexin(df_obs.Date, dates)
    observed_tanks = indexin(df_obs.Tank, tanks)
    Pcalc = Pcalc[CartesianIndex.(observed_tanks, observed_dates)]    
    return Pobs, Pcalc
end

function plot_cross(plots, df, tanks, dates; resolution=(600, 400), digits=2)
    
    f = Figure(; resolution)
    
    for (n, plt) ∈ enumerate(plots)
        # Измеренные и рассчитанные давления
        Pobs, Pcalc = observed_pressure(df, plt.col_name, tanks, dates, plt.calc_val)
        ρ = round(cor(Pobs, Pcalc); digits)
        
        # Максимальное значение на шкале
        max_val = cld(maximum(vcat(Pobs, Pcalc)), 100) * 100        
        
        # Оформление осей
        ax = Axis(f[1, n], ylabel="Pcalc, bar", xlabel="Pobs, bar", title=plt.title, aspect=1)
        xlims!(ax, 0, max_val)
        ylims!(ax, 0, max_val)
        
        # Отрисовка графиков
        lines!(ax, [0, max_val], [0, max_val], linewidth=2, color=:blue)
        lines!(ax, [0, max_val], [0, max_val*0.8], linewidth=2, color=:blue, linestyle=:dash)
        lines!(ax, [0, max_val], [0, max_val*1.2], linewidth=2, color=:blue, linestyle=:dash)
        scatter!(ax, Pobs, Pcalc, marker=:circle, markersize=10, color=:red, strokewidth=1)
        text!(ax, "ρ=$ρ",  position=(15, max_val-20), align=(:left, :center))
    end
    
    return f
end

cross_plots = (
    Pres = (col_name = :Pres, calc_val = prob.params.Pcalc, title = "Reservoir pressure"),
    Pbhp = (col_name = :Pbhp_prod, calc_val = prob.params.Pbhp, title = "BHP of producers"),
    Pinj = (col_name = :Pbhp_inj, calc_val = prob.params.Pinj, title = "BHP of injectors"),
)

f = plot_cross(cross_plots, df_rates, tanks, dates; resolution=(1000, 300))

f

## Решение обратной задачи

Для минимизации функции [(7)](#target-function) методами оптимизации, использующие информацию о производной, требуется вычислять градиент целевой функции. В модуле для этого используется метод сопряженных уравнений <a href=#ORL08 id=ORL08-link>[ORL08]</a>, в соответствии с которым градиент целевой функции вычисляется следующим образом

<a id='target-gradient'></a>
$$
\frac{d \beta}{d \mathbf m} = \nabla_\mathbf m \beta + \sum_{n=1}^L \left[ \nabla_\mathbf m \mathbf f^n \right]^\mathrm T \mathbf \mu^n \tag{14}
$$

где сопряженный вектор $\mu^n$ определяется путем решения системы линейных уравнений обратно во времени для $n = L,\, L-1,\, \ldots,\, 1$

<a id='adjoint-equation'></a>
$$
\left[ \nabla_{\mathbf P^n} \mathbf f^n \right] \mathbf \mu^n = -\left[ \nabla_{\mathbf P^n}{\mathbf f^{n+1}} \right] \mathbf \mu^{n+1} - \nabla_{\mathbf P^n} \beta \tag{15}
$$

<a id='terminal-condition'></a>
$$
\mathbf \mu^{L+1} = \mathbf 0 \tag{16}
$$

где $\left[ \nabla_{\mathbf P^n} \mathbf f^n \right]$ вычисляется в соответствии с [(6)](#system-jacobian) и

<a id='accumulation-jacobian'></a>
$$
\left[ \nabla_{\mathbf P^{n}} \mathbf f^{n+1} \right] = \frac{1}{\Delta t^{n}} \sum_{l = \mathrm w, \, \mathrm o} 
\operatorname{\mathbf D} \left[ -\left( \mathbf c_l + \mathbf c_{\mathrm f} \right) \circ 
\mathbf S_l^{n} \circ \frac{\mathbf V_{\mathrm p}^{n}}{\mathbf B_l^{n}} \right] \tag{16}
$$

In [ ]:
# Алгоритм расчета градиента целевой функции
adjoint_ = AdjointSolver{Float}(prob, targ, linalg, fset)

# Функция для вычисления значения и градиента целевой функции
function fun(fset::FittingSet, solver::NewtonSolver, targ::TargetFunction, adjoint::AdjointSolver)
    return (x, grad) -> begin
        setparams!(fset, x)
        solve!(solver)
        update_targ!(targ)
        if length(grad) > 0
            solve!(adjoint)
            copyto!(grad, adjoint.g)
        end
        return getvalue(targ)
    end    
end
optim_fun = fun(fset, solver, targ, adjoint_)

# Начальное значение параметров
initial_x = copy(getparams!(fset))
# Массив для хранения градиента
grad = similar(initial_x)

# Вычисление градиента методом конечных разностей
grad_fd = FiniteDiff.finite_difference_gradient(x -> optim_fun(x, Float[]), initial_x, Val(:central))

In [ ]:
# Вычисление градиента методом сопряженных уравнений
_ = optim_fun(initial_x, grad)
grad

In [ ]:
# Сравниваем градиенты, вычисленные различными способами
@test all(isapprox.(grad, grad_fd; rtol=0.0001))

In [ ]:
# Минимизация целевой функции с помощью пакета NLopt
opt = Opt(:LD_MMA, length(initial_x))
opt.lower_bounds = Float(-Inf)
opt.upper_bounds = Float(Inf)
opt.min_objective = optim_fun
opt.maxeval = 1000

minf, minx, ret = optimize(opt, initial_x)

numevals = opt.numevals
println("minf: $minf, numevals: $numevals, ret: $ret")

In [ ]:
# Сравнение значений целевой функции до и после минимизации
println("Before optimization:")
optim_fun(initial_x, [])
println(getvalues(targ))
calc_well_index!(targ)

println("After optimization:")
optim_fun(minx, [])
println(getvalues(targ))
calc_well_index!(targ)

In [ ]:
f = plot_cross(cross_plots, df_rates, tanks, dates; resolution=(1000, 300))
f

In [ ]:
# Графики по блокам
data_for_plots = (    
    Pbhp_calc = (getter = tank -> prob.params.Pbhp[parse(Int, tank[2]) + 1, :], 
                 axis = 2, plot = scatter!,
                 attrs = (marker=:rect, markersize=8, color=:cyan)),
    Pinj_calc = (getter = tank -> prob.params.Pinj[parse(Int, tank[2]) + 1, :], 
                 axis = 2, plot = scatter!,
                 attrs = (marker=:rect, markersize=8, color=:darkorange)),
    data_for_plots...
)

f = plot_tanks(tanks[2:end], data_for_plots, dates; resolution=(2000, 1000), ncol=3)
f

<a id=Ahm10 href=#Ahm10-link><strong>[Ahm10]</strong></a> T.H. Ahmed, Reservoir engineering handbook, 4th ed. Elsevier, 2010.

<a id=Dak01 href=#Dak01-link><strong>[Dak01]</strong></a> L.P. Dake, The Practice of Reservoir Engineering (Revised Edition), 1st ed., vol. 36, Elsevier Science, 2001.

<a id=EAKK01 href=#EAKK01-link><strong>[EAKK01]</strong></a> Turgay Ertekin, J.H. Abou-Kassem, and G.R. King, Basic Applied Reservoir Simulation, vol. 37, SPE Textbook Series, 2001.

<a id=Gol82 href=#Gol82-link><strong>[Gol82]</strong></a> T.D. van Golf-Racht, Fundamentals of Fractured Reservoir Engineering, 1st ed., Elsevier Science, 1982.

<a id=Lie19 href=#Lie19-link><strong>[Lie19]</strong></a> K.A. Lie, An Introduction to Reservoir Simulation Using MATLAB/GNU Octave: User Guide for the MATLAB Reservoir Simulation Toolbox (MRST), Cambridge University Press, 2019.

<a id=ORL08 href=#ORL08-link><strong>[ORL08]</strong></a> D.S. Oliver, A.C. Reynolds, and N. Liu Inverse Theory for Petroleum Reservoir Characterization and History Matching, Cambridge University Press, 2008.